# Logs

> Date: 18/03/25
> - Kanbn looks exceptionally promising for workflow!
>   - Though still getting used to the workflow involved in collaboration - I find the dynamics of teamwork difficult.
> - There were many temporary and personal files in the .Net build that needed to be gitignored. We eventually resolved the issue but on reflection we should have checked and added the necessary gitignore in the same commit as the build.
> - We also have realised that we need to investigate further how split our work and learners guide should be.
> - The .Net setup seems to have similarities to NodeJs which, considering there will be many things to learn, is a good base to start on.

> Date 03/02/25
> Debated pros and cons of using the advised flat file vs a database. Decided a database makes a whol lot more sense even if just using SQLite: Using a database means scaling will be very easy and may only need slight changes in syntax if we switch DB, a deciding reason is when I went through how the storage and oragnisation would work for each song having multiple words and their translations in a json flat file was gonna be very messy and hard to use, whereas a DB would be dynamically editable if we desired in the future (very flexible) and the organisation of words, their language and translations is just gonna be a lot easier to work with, though a little more effort to setup.
> Did lots of Kanbn task making trying to in precise detail figure out how all the systems will fit together exactly. 



> Date 04/02/25
> Microsoft Azure Setup
> Looked through the setup videos and page. It was very useful to find that we can manage permissions for the same resource group. It went through all the possible resources we could use, noting there is a translater, sql database and containers all within Azure. We realised we can likely do everything on Azure. Which can useful in the future if a company uses Azure, and the skills will be transferrable anyway.
> Installed Azure Tools Extension as part of Deploying a web app with database tutorial.
> https://portal.azure.com/?quickstart=true#view/Microsoft_Azure_Resources/ServicePage.ReactView/id/WebApp
> From that we found the link and it seems that when we do want to publish it it's fairly straightforward. 
> It seems to be set up very very well and Microsoft own Github so it's well set up for that too.

# Working with Github

## Command Line

- To amend a previous commit message: ```git commit --amend```
- To undo the last commit: ```git reset --soft HEAD~1```
- To gitignore the necessary folders and files: ```dotnet new gitignore```